In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

In [2]:
import nltk
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix

from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags

In [4]:
url="https://twitter.com/login"
#driver = webdriver.Chrome(executable_path=r"D:\chromedriver_win32\\chromedriver.exe")
driver = webdriver.Firefox(executable_path=r"D:\geckodriver.exe")
driver.get(url)


In [5]:
username=driver.find_element_by_xpath('//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[5]/label/div/div[2]/div/input')
username.send_keys('rupam_27'+Keys.ENTER)

In [6]:
userpass=driver.find_element_by_name("password")
userpass.send_keys('orop8t702'+Keys.ENTER)

In [7]:
search=driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/label/div[2]/div/input')
search.send_keys("pandemic birthrate"+Keys.ENTER)

In [8]:
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1  # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
n=1
test_list=[]
#while True:
while n<=100:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    n+=1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    test=driver.find_elements_by_xpath('//div[@class="css-1dbjc4n"]//div[@class="css-1dbjc4n"]//div[@lang="en"]')
    for j in range(len(test)):
        test_list.append(test[j].text)
    if (screen_height) * i > scroll_height:
        break 

In [9]:
test=driver.find_elements_by_xpath('//div[@class="css-1dbjc4n"]//div[@class="css-1dbjc4n"]//div[@lang="en"]')

In [10]:
text=pd.DataFrame(test_list,columns=['tweets'])

In [11]:
text.drop_duplicates(inplace=True,ignore_index=True)

In [12]:
text

,tweets
0,"NEW COVER: Because of the pandemic, half a mil..."
1,"""U.S. birthrate hits lowest ever"" are you tell..."
2,Britain is currently facing a ‘baby shortage’ ...
3,The amazing thing is that people think having ...
4,When Dr Parasaram answered a question re the b...
...,...
96,USA experienced a marginal decrease of 4% in b...
97,"""The spiraling costs of child care, health car..."
98,This is going to be very bad for America -> Be...
99,"The spiraling costs of child care, health care..."


In [13]:
### Create a Term-Document Matrix
# Remove stop words
stop = stopwords.words('english')
text = pd.DataFrame(text)
textcol = text['tweets']
textcol = textcol.apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [14]:
textcol

0      NEW COVER: Because pandemic, half million fewe...
1      "U.S. birthrate hits lowest ever" telling peop...
2      Britain currently facing ‘baby shortage’ – pan...
3      The amazing thing people think fucking pandemi...
4      When Dr Parasaram answered question birthrate ...
                             ...                        
96     USA experienced marginal decrease 4% birthrate...
97     "The spiraling costs child care, health care e...
98     This going bad America -> Because pandemic, ha...
99     The spiraling costs child care, health care ed...
100    Naw, really? “spiraling costs child care, heal...
Name: tweets, Length: 101, dtype: object

In [15]:
# Remove numerical values
patternnum = '\b[0-9]+\b'
textcol = textcol.str.replace(patternnum,'')
# Remove punctuation
patternpunc = '[^\w\s]'
textcol = textcol.str.replace(patternpunc,'')
# Convert to lowercase
textcol = textcol.apply(lambda x: " ".join(x.lower() for x in x.split()))
# Stem the words
porstem = PorterStemmer()
textcol = textcol.apply(lambda x: " ".join([porstem.stem(word) for word in x.split()]))

C:\Users\rups_\AppData\Local\Temp/ipykernel_12364/1843087779.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  textcol = textcol.str.replace(patternnum,'')
C:\Users\rups_\AppData\Local\Temp/ipykernel_12364/1843087779.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  textcol = textcol.str.replace(patternpunc,'')


In [16]:
#remove https
pat_http='^http?://'
textcol = textcol.str.replace(pat_http,'')

C:\Users\rups_\AppData\Local\Temp/ipykernel_12364/1908563498.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  textcol = textcol.str.replace(pat_http,'')


In [17]:
textcol

0      new cover becaus pandem half million fewer bab...
1      us birthrat hit lowest ever tell peopl want ba...
2      britain current face babi shortag pandem promp...
3      the amaz thing peopl think fuck pandem lower b...
4      when dr parasaram answer question birthrat i r...
                             ...                        
96     usa experienc margin decreas 4 birthrat fuck p...
97     the spiral cost child care health care educ al...
98     thi go bad america becaus pandem half million ...
99     the spiral cost child care health care educ gl...
100    naw realli spiral cost child care health care ...
Name: tweets, Length: 101, dtype: object

In [18]:
# Convert data into a document matrix
vectorizer = CountVectorizer()
tokens = pd.DataFrame(vectorizer.fit_transform(textcol).toarray(), columns=vectorizer.get_feature_names())
tokens.columns
print(tokens.columns.tolist())

['14', '146', '15', '158', '1588', '164', '1861', '19', '1918', '1970', '1980', '2017', '2019', '2020', '2021', '21', '247', '25', '300000', '36', '500k', '6th', '800000', '814', '840832', 'aaliyah', 'abbiekamin', 'abdl', 'abrupt', 'acceler', 'across', 'activist', 'actual', 'admit', 'adopt', 'affect', 'afford', 'after', 'aftermath', 'again', 'age', 'ajohnston12', 'aka', 'alabama', 'alarm', 'all', 'along', 'alreadi', 'alreadyfal', 'also', 'amaz', 'america', 'american', 'amid', 'among', 'and', 'answer', 'anvil', 'anyth', 'aphrodisiac', 'appear', 'arent', 'arous', 'articl', 'as', 'asahi', 'assum', 'assumpt', 'at', 'attent', 'attitud', 'avail', 'babi', 'babymak', 'back', 'bad', 'badli', 'bare', 'barkin', 'basic', 'be', 'bear', 'becaus', 'becom', 'befor', 'beg', 'begin', 'believ', 'benjamin', 'better', 'big', 'biggest', 'birth', 'birthrat', 'boom', 'born', 'both', 'bounc', 'bounceback', 'brain', 'breadbak', 'breed', 'brilliant', 'bring', 'britain', 'broader', 'brought', 'budget', 'build', '

In [19]:
# LDA
vectorizer = CountVectorizer(max_df=0.8, min_df=4, stop_words='english')
tweet_values = textcol.values.astype('U') #convert Panda values to unicode
doc_term_matrix = vectorizer.fit_transform(tweet_values)
doc_term_matrix.shape
LDA = LatentDirichletAllocation(n_components=4, random_state=35)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=4, random_state=35)

In [20]:

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['immigr', 'year', 'coronaviru', 'push', 'popul', 'rise', 'say', 'think', 'econom', 'declin']


Top 10 words for topic #1:
['year', 'ha', 'peopl', '2020', 'japan', 'acceler', 'covid19', 'birth', 'drop', 'declin']


Top 10 words for topic #2:
['becaus', 'boom', 'guarante', 'half', 'born', 'bounc', 'million', 'fewer', '2021', 'babi']


Top 10 words for topic #3:
['educ', 'spiral', 'climat', 'includ', 'global', 'cost', 'health', 'child', 'chang', 'care']




In [120]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['immigr', 'year', 'american', 'push', 'peopl', 'rise', 'think', 'econom', 'declin', 'say']


Top 10 words for topic #1:
['rate', 'year', 'ha', 'covid19', '2020', 'acceler', 'popul', 'birth', 'drop', 'declin']


Top 10 words for topic #2:
['becaus', 'boom', 'guarante', 'half', 'bounc', 'born', 'million', 'fewer', '2021', 'babi']


Top 10 words for topic #3:
['spiral', 'climat', 'includ', 'global', 'cost', 'health', 'child', 'coronaviru', 'chang', 'care']




In [122]:
text.to_csv("tweets on pandemic birth rate.csv")